# Common Task 2: Training and Evaluation of Transformer Model  

This notebook contains the solution for **Common Task 2**.  

## Training  
Refer to the bash scripts in the `runs/` directory.  
- Execute the corresponding `.sh` file to start training.  
- The default configuration is set for a **2-GPU system**.  
- Modify the parameters as needed based on your hardware.  

## Evaluation  
Follow the provided code in this notebook for testing.  

In [ ]:
from trainer import sequence_accuracy
from data import Data
import pandas as pd
import numpy as np
import os
import torch
from config import TransformerTestConfig
from tokenizer import Tokenizer
from fn_utils import get_model

In [ ]:
# Special tokens & coressponding ids
BOS_IDX, PAD_IDX, EOS_IDX, UNK_IDX, SEP_IDX = 0, 1, 2, 3, 4
special_symbols = ['<S>', '<PAD>', '</S>', '<UNK>', '<SEP>']

In [ ]:
config = TransformerTestConfig(
    model_name="transformer",
    root_dir="dummy_root",
    data_dir="dummy_data",
    device='cuda',
    embedding_size=512,
    nhead=8,
    num_encoder_layers=3,
    num_decoder_layers=3,
    dropout=0.1,
    hidden_dim=8192,
    src_max_len=280,
    tgt_max_len=323,
    src_voc_size=None,
    tgt_voc_size=None,
    seed=42,
    to_replace=False,
    index_pool_size=200,
    momentum_pool_size=200,
    is_prefix=False
)

In [ ]:
df_train = pd.read_csv(config.data_dir+"train.csv")
df_test = pd.read_csv(config.data_dir+"test.csv")
df_valid = pd.read_csv(config.data_dir+"valid.csv")

df = pd.concat([df_train,df_valid,df_test]).reset_index(drop=True)

In [ ]:
tokenizer = Tokenizer(df, config.index_pool_size,config.momentum_pool_size, special_symbols, UNK_IDX,config.to_replace)
src_vocab = tokenizer.build_src_vocab(config.seed)
src_itos = {value: key for key, value in src_vocab.get_stoi().items()}
tgt_vocab = tokenizer.build_tgt_vocab()
tgt_itos = {value: key for key, value in tgt_vocab.get_stoi().items()}

In [ ]:
config.src_voc_size = len(src_vocab)
config.tgt_voc_size = len(tgt_vocab)

In [ ]:
model = get_model(config)

In [ ]:
datasets = Data.get_data(
            df_train, df_test, df_valid, config, tokenizer,src_vocab, tgt_vocab)

test_ds = datasets['test']

In [ ]:
preds = sequence_accuracy(config,test_ds,tgt_itos,test_size=len(test_ds))